In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc; reload(fcc)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline

In [ ]:
reload(pu_delayfb)
carbons = [1,2,3,4,5,6,7]
LDE_attempts = [100] #, 100, 150, 200, 250, 500]
powers = [4e-6]
pulses = ["_pi2_"]#, "_pi_", "_none_"]
nt = "20170804080000"
ot = "20170805080000"
# nt = "20170808000000"
ot = None
trans = "_p1"

repump_time_data = dict()

for p in powers:
    p_key = int(p*1e6)
    repump_time_data[p_key] = dict()
    for l in LDE_attempts:
        l_key = l
        repump_time_data[p_key][l_key] = dict()
        for c in carbons:
            c_key = c
            repump_time_data[p_key][l_key][c_key] = dict()
            
            for pls in pulses:
                pls_key = pls.replace("_", "")
            
                print("Processing: power %d, LDE %d, carbon %d, pulse %s" % (p_key, l_key, c_key, pls_key))
                repump_time_data[p_key][l_key][c_key][pls_key] = None

                
#                 if c == 1 and l >101:
#                     continue

#                 if c != 7 and l > 450:
#                     continue

#                 if c == 7 and l < 201:
#                     continue

                try:
                    m_name = "Sweep_Repump_time%s%sC%d_LDE%d_pow%duW_X" % (trans if 'p1' in trans else '', pls, c, l, int(p*1e6))
                    data = dict()
                    data["x"], data["y"], data["y_u"], data["fr"] = \
                    pu_delayfb.average_repump_time(
                        contains = m_name,do_fit = True,fixed=[], ret_data_fit=True, 
                        newer_than=nt, older_than=ot,
                        title="avg repump time, \nC%d, LDE %d, power %duW, pulse %s" % (c, l, int(p*1e6), pls_key)
                    )
                    repump_time_data[p_key][l_key][c_key][pls_key] = data
                except Exception as e:
                    print("Data analysis failed: " + repr(e))

In [ ]:
plot_data = np.zeros((len(powers), len(LDE_attempts), len(carbons)))
plot_data_u = np.zeros_like(plot_data)

# these correspond to the measurement of 2-3-4 august 2017
# crappy_fits = [
#     (4, 50, 4),
#     (4, 50, 5),
#     (4, 150, 2),
#     (4, 200, 2),
#     (4, 250, 2),
#     (4, 250, 6),
#     (4, 250, 7),
#     (4, 500, 7),
#     (1, 100, 5),
#     (1, 250, 2),
#     (1, 250, 6),
#     (1, 250, 7),
#     (1, 500, 7),
# ]

# these correspond to the measurement of 4-5 august 2017 (-1 transition)
# crappy_fits = [
#     (4, 50, 5),
#     (4, 50, 2),
#     (4, 200, 1),
#     (4, 250, 1),
#     (4, 250, 5),
#     (4, 250, 7),
#     (4, 500, 7)
# ]

# okay_fits = [
#     (4,50,2),
#     (4,50,4),
#     (4,50,5),
#     (4,50,3),
#     (4,100,1),
#     (4,100,4),
#     (4,150,4),
#     (4,200,4)
# ]

pulse_to_analyse = 'pi2'

for i_p, p in enumerate(powers):
    p_key = int(p*1e6)
    for i_l, l in enumerate(LDE_attempts):
        l_key = l
        for i_c, c in enumerate(carbons):
            c_key = c
            
            plot_data[i_p, i_l, i_c] = np.nan
            plot_data_u[i_p, i_l, i_c] = np.nan
            
#             print("Processing: power %d, LDE %d, carbon %d" % (p_key, l_key, c_key))
#             if c == 1 and l >101:
#                 continue

#             if c != 7 and l > 450:
#                 continue

#             if c == 7 and l < 201:
#                 continue
                
#             if (p_key, l_key, c_key) in crappy_fits:
#                 print("Skipping crappy fit: " + str((p_key, l_key, c_key)))
#                 continue
                
#             if not (p_key, l_key, c_key) in okay_fits:
#                 print("Skipping crappy fit: " + str((p_key, l_key, c_key)))
#                 continue
                
            try: 
                plot_data[i_p, i_l, i_c] = repump_time_data[p_key][l_key][c_key][pulse_to_analyse]['fr']['params_dict']['x0']
                plot_data_u[i_p, i_l, i_c] = repump_time_data[p_key][l_key][c_key][pulse_to_analyse]['fr']['error_dict']['x0']
            except:
                print("No fitting data found for: %s, %s, %s, %s" % (p_key, l_key, c_key, pulse_to_analyse))

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)

couplings = []
deviations = []

for c in carbons:
    print ("Carbon: %d" % c)
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    ms1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1%s' % (c, trans)]
    avg_freq = msmt_params.cfg['samples']['111no2']['C%d_freq%s' % (c, trans)]
    
    parallel_coupling = ms1_freq - ms0_freq
    print("parallel coupling (kHz): \t%.3f" % (parallel_coupling*1e-3))
    
    couplings.append(parallel_coupling)
    
couplings = np.array(couplings) * 1e-3

In [ ]:
from analysis.lib.fitting import fit, common

plot_xvals = np.linspace(0,250,100)

for i_p, p in enumerate(powers):
    plt.figure()
    for i_c, c in enumerate(carbons[:-1]):
        ebar = plt.errorbar(LDE_attempts, plot_data[i_p,:,i_c]*1e3, plot_data_u[i_p,:,i_c]*1e3, fmt='o', 
                            label="C%d (%d kHz)" % (c, couplings[i_c]))
        
        g_a = 0.0
        g_b = 0.0
        p0, fitfunc, fitfunc_str = common.fit_line(g_a, g_b)
        
        clean_x = []
        clean_y = []
        
        dirty_y = plot_data[i_p,:,i_c]*1e3
        
        for i_clean in xrange(len(dirty_y)):
            if not np.isnan(dirty_y[i_clean]):
                clean_x += [LDE_attempts[i_clean]]
                clean_y += [dirty_y[i_clean]]
                
        clean_x = np.array(clean_x)
        clean_y = np.array(clean_y)

#         fit_result = fit.fit1d(clean_x, clean_y, None, p0=p0, fitfunc=fitfunc, do_print=True, fixed=[], ret=True)
#         a, b = fit_result['params']
#         plt.plot(plot_xvals, fit_result['fitfunc'](plot_xvals), color=ebar[0].get_color(),
#                  label='power %d, C%d (%d kHz), a=%.0f, b*100=%.0f' % (int(p*1e6), c, couplings[i_c], a, b*100)
#                 )
        
        

        
    lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))
    plt.xlabel("Number of LDE attempts (-)")
    plt.ylabel("Repump time offset (ns)")
#     plt.ylim(0,600)
#     plt.xlim(0,300)

In [ ]:
for i_p, p in enumerate(powers):
    plt.figure(figsize=(15,5))
    for i_l, l in enumerate(LDE_attempts):
        plt.errorbar(couplings, plot_data[i_p,i_l,:]*1e3, plot_data_u[i_p,i_l,:]*1e3, fmt='o', label='power %d, LDE %d' % (int(p*1e6), l))
        
    plt.legend()
#     plt.xlim(-100,400)
#     plt.ylim(0,600)
    plt.xlabel("Coupling strength (kHz)")
    plt.ylabel("Repump time offset (ns)")